# Initialize Everything

In [4]:
from pymongo import MongoClient
import pandas as pd

## Read MongoDB

In [5]:
client = MongoClient()
db = client["test"]
collection = db["matchapps"]

## Flatten, reorganize, and clean Data

In [6]:
df = pd.json_normalize(collection.find())

column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

df = df[column_order]
df

,_id,metadata.matchNumber,metadata.scouterName,metadata.robotTeam,metadata.robotPosition,leftStartingZone,climb,trapNotes,autoNotes.near,autoNotes.mid,autoNotes.far,autoNotes.amp,autoNotes.miss,teleNotes.near,teleNotes.mid,teleNotes.far,teleNotes.amp,teleNotes.miss
0,65d012a2ae8381234449007d,8,Jim,5773,red_1,True,park,1,1,4,2,0,2,3,3,3,3,0
1,65d012a2ae8381234449007f,21,Jim,7124,blue_1,False,none,1,3,4,3,4,1,0,2,2,0,1
2,65d012a2ae83812344490081,30,Jim,2399,red_3,True,source,1,4,3,4,2,1,4,4,1,1,1
3,65d012a2ae83812344490083,4,Jim,8645,blue_2,False,park,0,2,1,0,3,0,4,2,4,1,1
4,65d012a2ae83812344490085,27,Jim,2403,blue_2,True,park,1,4,0,4,2,3,2,4,3,2,0
5,65d012a2ae83812344490087,28,Jim,8645,red_2,True,source,1,0,2,2,1,3,4,3,3,4,1
6,65d012a2ae83812344490089,23,Jim,9246,blue_1,False,source,1,0,3,4,4,2,4,0,2,0,0
7,65d012a2ae8381234449008b,34,Jim,8173,red_1,False,amp,1,0,3,0,2,3,4,3,2,0,1
8,65d012a2ae8381234449008d,27,Jim,8173,blue_2,False,failed,1,2,1,4,4,3,0,3,0,3,0
9,65d012a2ae8381234449008f,21,Jim,5773,blue_2,False,center,1,2,2,1,1,1,0,0,2,4,0


## Initialize Output DF

In [7]:
output_df = pd.DataFrame()

# Function and Variable Initializations

## Complete List of all Teams in DF

In [8]:
team_list = df["metadata.robotTeam"].unique().tolist()

# Analyze Data

## Auto Data Analysis Calculations

### Auto Speaker Notes Average (near, mid, far)

In [9]:
output_df["Auto Speaker Notes Near Avg"] = df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

### Auto Amp Notes Average

In [10]:
output_df["Auto Notes Amp Avg"] = df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

### Auto Percent of Miss Notes

In [11]:
output_df["Auto Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = df[df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = df[df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + df[df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + df[df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + df[df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes

## Auto Data Analysis Calculations

### Tele Speaker Notes Average (near, mid, far)

In [12]:
output_df["Tele Speaker Notes Near Avg"] = df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

### Tele Amp Notes Average

In [13]:
output_df["Tele Notes Amp Avg"] = df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

### Tele Percent of Missed Notes

In [14]:
output_df["Tele Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = df[df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = df[df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + df[df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + df[df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + df[df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes

## Other Data Analysis Calculations

### Trap Notes

In [15]:
output_df["Trap Notes Avg"] = df.groupby("metadata.robotTeam")["trapNotes"].mean()

### Percent of Successful Climbs

In [16]:
output_df["Percent of Successful Climbs"] = None
for team in team_list:
    climb_list = df.loc[df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs

### Percent of Climbs that Harmonized

In [17]:
output_df["Percent of Harmonized Climbs"] = None

### Percent of Times Robot Left Starting Zone During Autos

In [18]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

# Finalize and Send Data

## Send Data

In [19]:
output_df
#df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)

,Avg Auto Speaker Notes Near,Avg Auto Speaker Notes Mid,Avg Auto Speaker Notes Far,Avg Auto Notes Amp,Auto Percent of Missed Notes,Percent of Missed Notes,Avg Tele Speaker Notes Near,Avg Tele Speaker Notes Mid,Avg Tele Speaker Notes Far,Avg Tele Notes Amp,Tele Percent of Missed Notes,Avg Trap Notes,Percent of Successful Climbs,Percent of Harmonized Climbs,Percent of Times Robot Left Starting Zone During Autos
metadata.robotTeam,,,,,,,,,,,,,,,
2303,2.200000,1.400000,1.600000,2.000000,None,1.363636,2.200000,1.800000,2.000000,1.800000,None,0.800000,1.0,None,0.400000
2399,1.888889,2.111111,2.666667,1.333333,None,0.653846,2.888889,2.333333,1.777778,2.666667,None,0.444444,0.857143,None,0.666667
2403,2.285714,1.857143,3.428571,1.714286,None,0.764706,2.428571,2.000000,2.000000,1.857143,None,0.571429,0.666667,None,0.714286
2723,2.333333,0.333333,1.000000,2.000000,None,1.750000,1.333333,2.666667,2.333333,0.666667,None,0.000000,1.0,None,0.000000
4181,1.750000,2.000000,1.250000,2.000000,None,0.636364,2.750000,3.250000,2.000000,3.250000,None,1.000000,0.5,None,0.750000
5773,2.000000,2.400000,1.000000,1.000000,None,0.888889,1.800000,2.600000,2.400000,2.400000,None,0.400000,1.0,None,0.400000
7124,1.750000,2.250000,1.750000,3.000000,None,1.200000,1.250000,2.000000,2.000000,2.500000,None,1.000000,0,None,0.500000
8173,1.800000,2.100000,1.900000,3.100000,None,1.300000,2.000000,2.700000,1.700000,1.800000,None,0.500000,0.857143,None,0.400000
8645,2.400000,1.800000,2.000000,2.200000,None,0.444444,3.600000,2.600000,2.200000,3.400000,None,0.600000,1.0,None,0.400000
